# get_patch_location_array_for_image_level(run_parameters)

In [1]:
import time
import os
import sys

from collections import OrderedDict
import argparse

import numpy as np
import pandas as pd
import yaml

from skimage.filters import threshold_otsu
from skimage.color import rgb2lab

from PIL import ImageDraw
from PIL import TiffImagePlugin as tip

import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

data_dir = '../../DigiPath_MLTK_data/Aperio'
file_type_list=['.svs', '.tif', '.tiff']
fs_od = get_file_size_ordered_dict(data_dir, file_type_list)
list_number = 0
for k, v in fs_od.items():
    print('%3i %30s: %i'%(list_number, k,v))
    list_number += 1

  0         CMU-1-Small-Region.svs: 1938955
  1               JP2K-33003-1.svs: 63847265
  2           CMU-1-JP2K-33005.svs: 132565343
  3                      CMU-1.svs: 177552579
  4                      CMU-3.svs: 253815723
  5               JP2K-33003-2.svs: 289250433
  6                      CMU-2.svs: 390750635


In [6]:
#                         test data file for threshold_otsu
run_parameters = {}

data_dir = '../data/images/' 
image_file_name = 'CMU-1-Small-Region.svs'
run_parameters['wsi_filename'] = os.path.join(data_dir, image_file_name)
print('Image File:\n', run_parameters['wsi_filename'])

run_parameters['thumbnail_divisor'] = 10
run_parameters['patch_select_method'] = 'threshold_otsu' # 'threshold_rgb2lab'
run_parameters['patch_height'] = 224
run_parameters['patch_width'] = 224
run_parameters['threshold'] = 0

"""
                output & time test:                CALL - - get_patch_location_array()
"""
t0 = time.time()
patch_location_array = get_patch_location_array(run_parameters)
tt = time.time() - t0

print('%i images found\t%0.3f s\n'%(len(patch_location_array), tt))

print('type(patch_location_array) =',type(patch_location_array), '\t', len(patch_location_array),'\n')
for d in patch_location_array:
    print(d)

Image File:
 ../data/images/CMU-1-Small-Region.svs
74 images found	0.287 s

type(patch_location_array) = <class 'list'> 	 74 

(28, 774)
(28, 998)
(28, 1222)
(252, 774)
(252, 998)
(252, 1222)
(252, 1446)
(476, 774)
(476, 998)
(476, 1222)
(476, 1446)
(476, 1894)
(700, 774)
(700, 998)
(700, 1222)
(700, 1446)
(924, 102)
(924, 326)
(924, 550)
(924, 774)
(924, 998)
(924, 1222)
(924, 1446)
(924, 1670)
(924, 1894)
(1148, 774)
(1148, 998)
(1148, 1222)
(1148, 1446)
(1148, 1670)
(1148, 1894)
(1372, 774)
(1372, 998)
(1372, 1222)
(1372, 1446)
(1372, 1670)
(1596, 774)
(1596, 998)
(1596, 1222)
(1596, 1446)
(1596, 1670)
(1820, 550)
(1820, 774)
(1820, 998)
(1820, 1222)
(1820, 1446)
(1820, 1670)
(2044, 550)
(2044, 774)
(2044, 998)
(2044, 1222)
(2044, 1446)
(2044, 1670)
(2044, 1894)
(2268, 550)
(2268, 774)
(2268, 998)
(2268, 1222)
(2268, 1446)
(2268, 1670)
(2492, 326)
(2492, 550)
(2492, 774)
(2492, 998)
(2492, 1222)
(2492, 1446)
(2492, 1670)
(2716, 326)
(2716, 550)
(2716, 774)
(2716, 998)
(2716, 1222)
(

In [8]:
def get_patch_location_array_for_image_level(run_parameters):
    """ Usage: patch_location_array = get_patch_location_array_for_image_level(run_parameters)
        using 'patch_select_method", find all upper left corner locations of patches
        that won't exceed image size givin the 'patch_height' and 'patch_width'

    Args (run_parameters):  python dict.keys()
                                wsi_filename:           file name (with valid path)
                                patch_height:           patch size = (patch_width, patch_height)
                                patch_width:            patch size = (patch_width, patch_height)
                                thumbnail_divisor:      wsi_image full size divisor to create thumbnail image
                                patch_select_method:    'threshold_rgb2lab' or 'threshold_otsu'
    Returns:
        patch_location_array

    """
    patch_location_array = []

    wsi_filename = run_parameters['wsi_filename']
    thumbnail_divisor = run_parameters['thumbnail_divisor']
    patch_select_method = run_parameters['patch_select_method']
    threshold = run_parameters['threshold']
    patch_height = run_parameters['patch_height']
    patch_width = run_parameters['patch_width']
    image_level = run_parameters['image_level']

    #                     OpenSlide open                      #
    os_im_obj = openslide.OpenSlide(wsi_filename)
    obj_level_diminsions = os_im_obj.level_dimensions

    pixels_height = obj_level_diminsions[image_level][1]
    rows_fence_array = get_fence_array(patch_length=patch_height, overall_length=pixels_height)

    pixels_width = obj_level_diminsions[image_level][0]
    cols_fence_array = get_fence_array(patch_length=patch_width, overall_length=pixels_width)

    thumbnail_size = (pixels_width // thumbnail_divisor, pixels_height // thumbnail_divisor)
    small_im = os_im_obj.get_thumbnail(thumbnail_size)
    os_im_obj.close()
    #                     OpenSlide close                     #

    mask_im = get_sample_selection_mask(small_im, patch_select_method)

    it_rows = zip(rows_fence_array[:, 0] // thumbnail_divisor,
                  rows_fence_array[:, 1] // thumbnail_divisor,
                  rows_fence_array[:, 0])

    lft_cols = cols_fence_array[:, 0] // thumbnail_divisor
    rgt_cols = cols_fence_array[:, 1] // thumbnail_divisor
    cols_array = cols_fence_array[:, 0]

    for tmb_row_top, tmb_row_bot, row_n in it_rows:
        it_cols = zip(lft_cols, rgt_cols, cols_array)
        for tmb_col_lft, tmb_col_rgt, col_n in it_cols:
            if (mask_im[tmb_row_top:tmb_row_bot, tmb_col_lft:tmb_col_rgt]).sum() > threshold:
                patch_location_array.append((row_n, col_n))

    return patch_location_array



In [9]:
run_parameters['image_level'] = 0
patch_location_array = get_patch_location_array_for_image_level(run_parameters)
print(len(patch_location_array))

74
